#### Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:
*   Alumno 4: 






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será [Acrobot_V1](https://gymnasium.farama.org/environments/classic_control/acrobot/) y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será **alcanzar el objetivo de altura durante 100 episodios de test consecutivos**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [3]:
import sys
from pathlib import Path
import json
import numpy as np
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display

ROOT = Path('.').resolve()
sys.path.append(str(ROOT / 'src'))

from acrobot_dqn.plots import plot_training_logs, plot_eval_results


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Hot reload de módulos locales (evita reiniciar kernel)
%load_ext autoreload
%autoreload 2


#### Configuración base

In [ ]:
# Identificador del run que quieres reportar
RUN_ID = 'dqn_base_01'

RUN_DIR = Path('outputs') / 'runs' / RUN_ID
CONFIG_PATH = RUN_DIR / 'config.yaml'
EVAL_PATH = RUN_DIR / 'metrics' / 'eval.json'
FIG_DIR = RUN_DIR / 'figures'

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f'No se encontro {CONFIG_PATH}. Asegura que el run existe.')

cfg = yaml.safe_load(CONFIG_PATH.read_text(encoding='utf-8'))
ENV_NAME = cfg.get('env_name', 'Acrobot-v1')
LOG_PATH = RUN_DIR / 'logs' / f'dqn_{ENV_NAME}_log.json'


In [ ]:
# Resumen del modelo (si existe el .keras del run)
MODEL_PATH = Path('outputs') / 'weights' / f'{RUN_ID}_model.keras'
if MODEL_PATH.exists():
    try:
        import tensorflow as tf
        model = tf.keras.models.load_model(MODEL_PATH)
        model.summary()
    except Exception as e:
        print(f'No se pudo cargar el modelo: {e}')
else:
    print(f'No se encontro el modelo: {MODEL_PATH}')
    print('Resumen aproximado desde config:')
    print('Hidden units:', cfg.get('model', {}).get('hidden_units'))
    print('Activation:', cfg.get('model', {}).get('activation'))


### Arquitectura de clases (resumen)

- **DQNModelBuilder**: define la arquitectura MLP para aproximar $Q(s,a)$.
- **DQNAgentFactory**: construye el agente DQN con replay buffer, política $\epsilon$-greedy y variantes.
- **DQNTrainer**: ejecuta el entrenamiento, guarda logs, checkpoints y figuras.
- **DQNEvaluator**: evalúa en 100 episodios, guarda métricas y genera gráficas.


### Ejecución recomendada (AWS) y flujo reproducible

**1) Entrenar en AWS (base):**
```bash
python scripts/train.py --config configs/dqn_base.yaml --run-id dqn_base_01
```
Esto crea:
- `outputs/runs/dqn_base_01/` (config + logs + figuras)
- `outputs/weights/dqn_base_01_weights.h5f` (pesos)
- `outputs/weights/dqn_base_01_model.keras` (modelo completo, opcional)

**2) Evaluar en AWS (100 episodios):**
```bash
python scripts/eval.py --config configs/dqn_base.yaml \
  --weights outputs/weights/dqn_base_01_weights.h5f \
  --episodes 100 \
  --save outputs/runs/dqn_base_01/metrics/eval.json
```
Las figuras de evaluación se guardan en `outputs/runs/dqn_base_01/figures/`.

**3) Traer a local (git o scp):**
- `outputs/runs/dqn_base_01/`
- `outputs/weights/dqn_base_01_weights.h5f`

**4) Documentar en el notebook (local):**
- Ajusta `CONFIG_PATH` y `weights_path` con el `run_id` usado.
- Ejecuta las celdas de evaluación y gráficas.


In [ ]:
def set_global_seeds(seed=SEED):
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_global_seeds(SEED)


1. Implementación de la red neuronal

La red para **Acrobot** no trabaja con píxeles, sino con el vector de estado continuo de 6 dimensiones. Por ello se utiliza una **MLP** sencilla que aproxima la función de valor de acción $Q_	heta(s,a)$, siguiendo la formulación de DQN [1]. En esta arquitectura, el estado se aplana y pasa por dos capas densas con activación ReLU; la salida produce un valor por acción discreta.

La elección de una MLP (en lugar de CNN) se justifica porque el espacio de observaciones no contiene estructura espacial, y el objetivo es aproximar $Q(s,a)$ de forma estable y eficiente [1], [2].


In [1]:
# Mostrar hiperparámetros usados en el run
config_df = pd.json_normalize(cfg, sep='.')
config_df = config_df.T.rename(columns={0: 'value'})
display(config_df)


NameError: name 'DQNModelBuilder' is not defined

2. Implementación de la solución DQN

Se implementa DQN con: (i) **replay buffer** para romper correlaciones, (ii) política $\epsilon$-greedy con *annealing*, y (iii) *target network* opcionalmente actualizado cada cierto número de pasos, lo cual estabiliza el aprendizaje [1]. Las variantes **Double DQN** y **Dueling DQN** se activan con banderas, siguiendo las propuestas de estabilización y mejora del estimador de $Q$ descritas en [1].

La función objetivo se basa en el target de Bellman:

$$y(s,a)=r + \gamma \max_{a'} Q_{\theta^-}(s',a')$$

y la pérdida se minimiza típicamente con MSE o Huber:

$$\mathcal{L}(\theta)=\mathbb{E}\left[(y(s,a)-Q_{\theta}(s,a))^2\right].$$



In [ ]:
# Graficas de entrenamiento (se generan a partir del log)
if LOG_PATH.exists():
    plot_training_logs(LOG_PATH, FIG_DIR)
    for name in ['training_reward.png', 'training_mean_q.png']:
        img_path = FIG_DIR / name
        if img_path.exists():
            display(Image(filename=str(img_path)))
else:
    print(f'No se encontro el log: {LOG_PATH}')


In [ ]:
# Cargar resultados de evaluación
if EVAL_PATH.exists():
    results = json.loads(EVAL_PATH.read_text(encoding='utf-8'))
    display(pd.DataFrame([{
        'successes': results.get('successes'),
        'success_rate': results.get('success_rate'),
        'mean_reward': results.get('mean_reward'),
    }]))
    plot_eval_results(results, FIG_DIR)
    for name in ['eval_rewards.png', 'eval_rewards_hist.png']:
        img_path = FIG_DIR / name
        if img_path.exists():
            display(Image(filename=str(img_path)))
else:
    print(f'No se encontro el archivo de evaluacion: {EVAL_PATH}')


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

**Justificación del diseño**

- **Arquitectura MLP**: Acrobot provee un estado continuo de baja dimensión; por tanto, una MLP es suficiente para aproximar $Q(s,a)$ sin necesidad de convoluciones [1].
- **Replay buffer**: reduce correlación temporal entre muestras y mejora la estabilidad del aprendizaje [1].
- **Target network**: desacopla la red objetivo de la red online, reduciendo oscilaciones [1].
- **Double/Dueling DQN (opcional)**: Double DQN reduce la sobreestimación de $Q$; Dueling separa el valor del estado y la ventaja por acción [1].

**Resultados (completar con la ejecución real)**

- Recompensa media en test (100 episodios): ________
- Éxitos (meta alcanzada): ________/100
- Observaciones sobre estabilidad, velocidad de convergencia y sensibilidad a hiperparámetros.

**Referencias (IEEE)**

[1] A. Géron, *Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow*, 3rd ed., 2025, ch. 18.

[2] R. S. Sutton and A. G. Barto, *Reinforcement Learning: An Introduction*, 2nd ed., MIT Press, 2020.


---